<a href="https://colab.research.google.com/github/Saagar2005/CS203/blob/main/CS253.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.drop(['ID','Candidate','Constituency ∇'],axis = 1, inplace = True)
test_data.drop(['ID','Candidate','Constituency ∇'],axis = 1, inplace = True)

In [4]:
def convert_to_numeric(amount_str):
    multiplier = 1
    if 'Lac+' in amount_str:
        multiplier = 10 ** 5
    elif 'Crore+' in amount_str:
        multiplier = 10 ** 7
    elif 'Thou+' in amount_str:
        multiplier = 1000

    numeric_part = int(amount_str.split()[0])

    return numeric_part * multiplier

In [5]:
train_data['Total Assets'] = train_data['Total Assets'].apply(convert_to_numeric)
train_data['Liabilities'] = train_data['Liabilities'].apply(convert_to_numeric)
test_data['Total Assets'] = test_data['Total Assets'].apply(convert_to_numeric)
test_data['Liabilities'] = test_data['Liabilities'].apply(convert_to_numeric)

In [6]:
from sklearn.preprocessing import LabelEncoder
party_label_encoder = LabelEncoder()
train_data['Party'] = party_label_encoder.fit_transform(train_data['Party'])
test_data['Party'] = party_label_encoder.transform(test_data['Party'])

In [7]:
state_label_encoder = LabelEncoder()

train_data['state'] = state_label_encoder.fit_transform(train_data['state'])
test_data['state'] = state_label_encoder.transform(test_data['state'])

In [8]:
from sklearn.preprocessing import OrdinalEncoder

education_order = ['Literate','5th Pass','8th Pass','10th Pass','12th Pass','Graduate','Graduate Professional','Post Graduate','Doctorate','Others']
ordinal_encoder = OrdinalEncoder(categories=[education_order])
train_data['Education'] = ordinal_encoder.fit_transform(train_data[['Education']])

In [9]:
'''import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(train_data['Liabilities'], train_data['Total Assets'], train_data['Education'], c='r', marker='o')

# Set labels and title
ax.set_xlabel('Criminal Case')
ax.set_ylabel('Total Assets')
ax.set_zlabel('Education')
ax.set_title('3D Scatter Plot')

# Optionally, use Seaborn for styling
sns.set(style="whitegrid")'''

'import matplotlib.pyplot as plt\nfrom mpl_toolkits.mplot3d import Axes3D\nimport seaborn as sns\n%matplotlib inline\n\nfig = plt.figure()\nax = fig.add_subplot(111, projection=\'3d\')\n\nax.scatter(train_data[\'Liabilities\'], train_data[\'Total Assets\'], train_data[\'Education\'], c=\'r\', marker=\'o\')\n\n# Set labels and title\nax.set_xlabel(\'Criminal Case\')\nax.set_ylabel(\'Total Assets\')\nax.set_zlabel(\'Education\')\nax.set_title(\'3D Scatter Plot\')\n\n# Optionally, use Seaborn for styling\nsns.set(style="whitegrid")'

In [10]:
train_y = train_data['Education']
train_X = train_data.drop(['Education'], axis = 1)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.metrics import f1_score

# Define the candidate kernels and their hyperparameters
kernels = [
    1.0 * RBF(1.0),
    1.0 * Matern(length_scale=1.0, nu=1.5),
    1.0 * WhiteKernel(noise_level=1.0)
]

# Create a Gaussian process classifier
gpc = GaussianProcessClassifier()

# Set up the parameter grid for grid search
param_grid = {'kernel': kernels}

# Perform grid search with cross-validation to find the best kernel
grid_search = GridSearchCV(gpc, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Get the best kernel and its hyperparameters
best_kernel = grid_search.best_estimator_.kernel_
best_params = grid_search.best_params_

# Fit the GPC with the best kernel and hyperparameters
best_gpc = GaussianProcessClassifier(kernel=best_kernel).fit(X_train, y_train)

# Evaluate the F1 score on the test set
y_pred = best_gpc.predict(X_test)
f1_score = f1_score(y_test, y_pred)

print(f"Best Kernel: {best_kernel}")
print(f"Best Parameters: {best_params}")
print(f"F1 Score: {f1_score}")

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:430: Converg

In [ ]:
#from sklearn.metrics import f1_score

#f1 = f1_score(y_test, predictions, average='weighted')
#f1